In [1]:
!pip install langid


In [2]:
import requests
import os
import langid
import tensorflow as tf

from random import randint as ran
from math import floor
from bs4 import BeautifulSoup

In [3]:
urls = []
used_urls = []
text_File = open("text.txt","a")

In [ ]:


def get_text(url):
  global used_urls

  temp_urls = []
  texts = []
  try:
    content = requests.get(url)
    #print(url)
  except:
    return []
  soup = BeautifulSoup(requests.get( url ).content, 'html.parser')
  used_urls.append(url)

  # get text from soup
  for para in soup.findAll("p"):
    text = para.get_text()
    text = " ".join(text.split())
  # add to a list and then write list to file so we can use the "not in" checks
    if text not in texts and text not in [""," "] and langid.classify(text)[0] == "en":
      texts.append(text)
    for t in texts:
      if t != "" and len(t.split()) > len("five_") and t[-1] != ":":
        try:
            text_File.write(t + "\n")
        except:
            pass
            #print("cant write")
    texts = []

  # get urls
  for link in soup.findAll("a"):
    # get the link and onwards from the text
    linkText = str(link)
    startIndex = linkText.find("href")

    # if has link
    if startIndex != -1:
      link_and_stuff = linkText[startIndex + len('href="')::]
      
  # if link belongs to wikipedia, 
  #we add the wikilink because paths are relative, 
  #in theory we could do this by code with every website
  # but im lazy
      # link_and_stuff.startswith('/wiki/')
      if link_and_stuff.startswith('/wiki/'):
        WIKILINK = "https://en.wikipedia.org"
        link_and_stuff = WIKILINK + link_and_stuff
      
      # remove the onwards from the text
      endIndex = 0

      if link_and_stuff[0:len("http")] != "http":
        pass
      elif "'" in link_and_stuff:
        endIndex = link_and_stuff.find("'")
      elif '"' in link_and_stuff:
        endIndex = link_and_stuff.find('"')

      url_ = link_and_stuff[0:endIndex]
      if url_ != "" and url_ not in used_urls and url_ not in temp_urls and url_ not in urls:
        temp_urls.append(url_)
  
  return temp_urls

def main():
  urls = ["https://en.wikipedia.org/wiki/Main_Page"]
  THRESHOLDINGB = 3.5

  while True:
    fileSize = float(os.path.getsize("text.txt")  / (1024 ** 3)) # in bytes so convert to gb
    #print(str(floor(fileSize * 10000)/10000) + "GB") #filesize formatted

    if len(urls) == 0:
      urls.append("https://en.wikipedia.org/wiki/Special:Random")

    if fileSize < THRESHOLDINGB:
      for i in range(32):
        
        #print("urls",len(urls))
        _ = get_text(urls[0])

        if len(urls) < 50:
          urls.extend(_)
        else:
            pass  
            #print("urls too large")

        urls.pop(0)

    else:
      print("File Done!")
      return

  print("used urls:\n      ",used_urls)

if __name__ == "__main__":
  main()